In [11]:
# %matplotlib inline
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.signal as signal
import seaborn as sns
import os.path as op
import skfda
from sklearn.decomposition import NMF
import tslearn
import networkx
from glob import glob

In [12]:
def make_emg_obj(fname):
  dat = sp.io.loadmat(fname)
  dat_seq = dat["data"]
  st = dat["datastart"]
  ed = dat["dataend"]
  labels = dat["titles"]
  dat_emg = pd.DataFrame()
  for s, e, la in zip(st, ed, labels):
    if s == -1:
      continue
    else:
      la = la.replace(" ", "")
      dat_emg[la] = dat_seq[0][int(s-1):int(e-1)]
  if dat_emg.shape[1] == 16:
    labels = ["Rt_TA", "Rt_SOL", "Rt_GL", "Rt_GM", "Rt_VM", "Rt_VL", "Rt_Ham", "Lt_TA", 
              "Lt_SOL", "Lt_GL", "Lt_GM", "Lt_VM", "Lt_VL", "Lt_Ham", "Rt_foot", "Lt_foot"]
    dat_emg.columns = labels
    val = dat_emg["Rt_foot"].values
    val = val - min(val)
    val[val < np.max(val)/3] = 0
    events = [1 if val[j-1] == 0 and val[j]>0 else 0 for j in range(len(val))]
    dat_emg_foot = dat_emg.iloc[:,:14]
    dat_emg_foot["Foot"] = events
  else:
    val = dat_emg["Foot"].values
    val = val - min(val)
    val[val < np.max(val)/3] = 0
    events = [1 if val[j-1] == 0 and val[j]>0 else 0 for j in range(len(val))]
    dat_emg_foot = dat_emg
    dat_emg_foot["Foot"] = events
  return dat_emg_foot



# set montage
def make_montage(fname):
  montage = mne.channels.read_custom_montage(fname)
  _montage = montage.get_positions()["ch_pos"]
  
  for mtg in _montage:
    _montage[mtg] += (0, 0.01, 0.04)
  
  return mne.channels.make_dig_montage(_montage)



def read_eeg(fname, montage):
  raw = mne.io.read_raw_edf(fname)
  ch_names = raw.info["ch_names"]
  new_names = [ch_name.replace("EEG ","").replace("-Ref","") for ch_name in ch_names]
  ch_names_dic = dict(zip(ch_names, new_names))
  mne.rename_channels(raw.info, ch_names_dic)
  retype = {"EOG":"eog"}
  raw.set_channel_types(retype)
  raw.set_montage(montage)
  return raw.set_montage(montage)



def add_event(eeg, emg):
  length = min(eeg.last_samp, len(emg)-1)
  foot = emg.loc[:length, ["Foot"]].values.T
  stim = mne.create_info(ch_names = ["Heel"], sfreq=1000, ch_types = "stim")
  event = mne.io.RawArray(data = foot, info = stim)
  eeg.crop(0, length/1000)
  eeg.load_data()
  return eeg.add_channels([event]), emg.iloc[:(length+1),:]



def prep_eeg(eeg, l_freq, h_freq, method, bad_channels):
  eeg.load_data()  # データを読み込む
  eeg.filter(l_freq=l_freq, h_freq = h_freq, method = method)
  eeg.info["bads"].extend(bad_channels)  # bad channels
  return eeg.set_eeg_reference(ref_channels='average', projection=True)



def prep_emg(emg, h_freq, l_freq, Wn, btype):
  emg_raw = emg.iloc[:,:10]
  emg_event = emg["Foot"]
  event_idx = emg_event[emg_event==1].index
  emg_raw = emg_raw - emg_raw.mean()
  b, a = sp.signal.butter(Wn, [h_freq,l_freq], btype=btype)
  for ch in emg_raw.columns:
    emg_raw[ch] = sp.signal.filtfilt(b, a, emg_raw[ch])
  return  emg_raw.abs(), event_idx



# エポッキング
def make_epochs(eeg, emg, tmin, tmax):
  event = mne.find_events(eeg)
  event_dict = {"HC":1}
  eeg_epochs = mne.Epochs(eeg, event, event_id = event_dict, tmin = tmin, tmax = tmax, preload = True)
  # eeg_epochs.drop_log
  emg_drop_log = []
  # EMGエポッキング
  for i, idx in enumerate(event_idx):
    if i == len(event_idx):
      break
    elif len(emg_raw.iloc[int(idx+tmin*1000):int(idx+tmax*1000),:]) < (tmax -tmin)*1000:
      emg_drop_log.append(f"the {i+1}th epoch has too short! exclude this epoch. {len(emg_raw.iloc[int(idx+tmin*1000):int(idx+tmax*1000),:])}")
      continue
    elif i == 0:
      emg_epochs = emg_raw.iloc[int(idx+tmin*1000):int(idx+tmax*1000),:].values[np.newaxis]
    else:
      emg_epochs = np.append(emg_epochs, emg_raw.iloc[int(idx+tmin*1000):int(idx+tmax*1000),:].values[np.newaxis], axis=0)
  return eeg_epochs, emg_epochs, eeg_epochs.drop_log, emg_drop_log



def make_epochs_list(eeg, emg, event_idx):
  emg_epochs = []
  for ev in range(len(event_idx)):
    if ev+1 == len(event_idx):
      break
    else:
      emg_epochs.append(emg_raw.iloc[event_idx[ev]:event_idx[ev+1]])
  eeg_raw = pd.DataFrame(eeg.get_data().T, columns= eeg.info["ch_names"]).iloc[:,:64]
  eeg_epochs = []
  for ev in range(len(event_idx)):
      if ev+1 == len(event_idx):
          break
      else:
          eeg_epochs.append(eeg_raw.iloc[event_idx[ev]:event_idx[ev+1]])
  return eeg_epochs, emg_epochs


# LLN
def approx(x, method, n):
  import numpy as np
  from scipy.interpolate import interp1d
  y = np.arange(0, len(x), 1)
  f = interp1d(y, x, kind = method)
  y_resample = np.linspace(0, len(x)-1, n)
  return f(y_resample)


def lln_list(epochs_list, n):
  #n = n
  method = "linear"
  epochs = np.zeros([len(epochs_list), n, epochs_list[0].shape[1]])
  for i, epoch in enumerate(epochs_list):
    for j, mus in enumerate(epoch):
      if i == len(epochs_list):
        break
      epochs[i,:,j] = approx(epoch[mus], method, n)
  return epochs


In [13]:
def make_emg_epochs(emg_raw, event_idx):
    emg_epochs = []
    for ev in range(len(event_idx)):
        if ev+1 == len(event_idx):
            break
        else:
            emg_epochs.append(emg_raw.iloc[event_idx[ev]:event_idx[ev+1]])

In [14]:
emg_file_path = glob("../data/Data2022/edf/EMG/Tsutsumi*.mat")
emg_file_path.sort()
emg_file_path

['../data/Data2022/edf/EMG/Tsutsumi_RAS100.mat',
 '../data/Data2022/edf/EMG/Tsutsumi_RAS110.mat',
 '../data/Data2022/edf/EMG/Tsutsumi_RAS90.mat',
 '../data/Data2022/edf/EMG/Tsutsumi_noRAS.mat']

In [15]:
dat = make_emg_obj(emg_file_path[3])

## EMG前処理
- サンプリングレート2000Hzでデータを取得し、5〜400Hzでバンドパス
- 20Hzハイパスフィルタリング、ヒルベルト変換で整流（Boonstra, 2016）

## 筋間コヒーレンス
- 窓長1秒、オーバーラップ0.75秒
- 各条件で平均化し、二乗して二乗コヒーレンスとする



#### 前処理
1. 5〜400Hzバンドパスフィルター

In [7]:
N = 4
h_freq = 5
l_freq = 200
fs = 1000
btype = "bandpass"
raw = dat.iloc[:,:10]
emg_event = dat["Foot"]
event_idx = emg_event[emg_event==1].index
raw = raw - raw.mean()
b, a = sp.signal.butter(N=N, Wn=[h_freq,l_freq], btype=btype, fs=fs)
for ch in raw.columns:
    raw[ch] = sp.signal.filtfilt(b, a, raw[ch])
raw

,TA_prox,TA_dist,SOL,GL,GM,RF,VM,VL,Ham_med,Ham_lat
0,-0.002373,0.001533,0.000777,-0.001572,0.000730,0.000571,0.001125,0.000799,0.000023,0.001426
1,-0.003657,0.001559,0.003243,-0.003058,-0.000587,0.001322,-0.001941,-0.000456,0.000523,0.002829
2,-0.006858,0.002808,0.008096,-0.004558,-0.001456,0.000516,-0.005151,-0.002501,0.000348,0.003511
3,-0.011177,0.005141,0.014298,-0.006390,-0.002092,-0.001253,-0.008187,-0.004669,-0.000604,0.003828
4,-0.014281,0.007167,0.018696,-0.008483,-0.002675,-0.001912,-0.010427,-0.005731,-0.001920,0.004480
...,...,...,...,...,...,...,...,...,...,...
304594,-0.000130,-0.003000,0.012553,-0.008011,-0.003745,-0.000324,-0.000082,0.003414,0.001716,-0.001125
304595,-0.000912,-0.002311,0.003928,-0.006871,-0.003728,0.000661,0.000325,0.002716,0.001223,-0.000492
304596,-0.001755,0.000188,-0.002421,-0.004276,-0.003684,0.001706,0.000028,0.001862,0.001081,0.001106
304597,-0.001388,0.001288,-0.002002,-0.001834,-0.002457,0.001457,-0.000080,0.001108,0.000943,0.001549


2. 20Hzハイパスフィルタリング、ヒルベルト変換（エンベロープ）

In [8]:
N = 4
h_freq = 20
btype = "highpass"

b, a = sp.signal.butter(N=N, Wn=h_freq, btype=btype, fs=fs)
for ch in raw.columns:
    raw[ch] = sp.signal.filtfilt(b, a, raw[ch])
    raw[ch] = sp.signal.hilbert(raw[ch])
raw = np.abs(raw)
print(raw.min())
raw

TA_prox    0.000010
TA_dist    0.000005
SOL        0.000016
GL         0.000003
GM         0.000003
RF         0.000001
VM         0.000009
VL         0.000004
Ham_med    0.000006
Ham_lat    0.000005
dtype: float64


,TA_prox,TA_dist,SOL,GL,GM,RF,VM,VL,Ham_med,Ham_lat
0,0.002700,0.001144,0.008476,0.002602,0.000431,0.000704,0.005421,0.004642,0.000222,0.001313
1,0.005017,0.001696,0.009944,0.003367,0.000952,0.001413,0.007880,0.005156,0.000879,0.001533
2,0.007437,0.002790,0.012596,0.004367,0.001623,0.002207,0.008954,0.006145,0.001486,0.001445
3,0.009322,0.003117,0.014816,0.005331,0.002121,0.002264,0.010016,0.006615,0.001955,0.001393
4,0.010074,0.003237,0.015370,0.006410,0.002740,0.001798,0.010355,0.006547,0.002269,0.001774
...,...,...,...,...,...,...,...,...,...,...
304594,0.002104,0.004362,0.009048,0.009925,0.005362,0.000725,0.001396,0.002889,0.000949,0.002241
304595,0.003195,0.003973,0.008293,0.007777,0.004781,0.001547,0.002314,0.003033,0.000691,0.001971
304596,0.003406,0.003031,0.008676,0.005353,0.004570,0.002169,0.002870,0.003196,0.000425,0.001535
304597,0.002632,0.001115,0.010020,0.003186,0.004316,0.002358,0.003160,0.003230,0.000419,0.000714


In [9]:
emg_epochs = []
for ev in range(len(event_idx)):
    if ev+1 == len(event_idx):
        break
    else:
        emg_epochs.append(raw.iloc[event_idx[ev]:event_idx[ev+1],:])

## コヒーレンス解析
### [scipy.signal.coherence(x, y, fs, window, nperseg, noverlap, nfft, detrend, axis)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.coherence.html#scipy.signal.coherence)

Parameters
- x, y: 信号
- fs: サンプリング周波数
- window: array-likeで与える場合、window幅はそのまま利用される（長さはnpersegでなければならない）
- nperseg: 各セグメンテーションの長さ。Noneの場合は256に設定され、windowがarray-likeの場合はその長さが与えられる。
- noverlap: 窓のオーバーラップ。もし指定がなければ、nperseg//2で与えられる。
- nfft: ゼロパディングされたFFTが必要な場合に使用されるFFTの長さ。指定がなければnpersegとなる。

Returns
- f: コヒーレンスを計算する周波数軸
- Cxy: コヒーレンスの大きさ（1D; ndarray）

In [10]:
nperseg = 512
res = np.zeros([int(nperseg/2)+1, 10,10])
resBig = np.ndarray((1,int(nperseg/2)+1,10,10))
for df in emg_epochs:
    for i,la1 in enumerate(df):
        for j,la2 in enumerate(df):
            x = df[la1]
            y = df[la2]
            f, Cxy = signal.coherence(y, x, fs=1000, nperseg=nperseg)
            res[:,i,j] = Cxy
    resBig = np.append(resBig, res[np.newaxis],axis=0)

KeyboardInterrupt: 

In [ ]:
a = 8<=f
b = f<13
alpha = a==b  # alpha
a = 13<=f
b = f < 26
beta = a==b  #beta
a = 26<=f
b = f < 30
gamma = a==b  #gamma
a = 0.5<=f
b = f < 4
delta = a==b  #delta
a = 4<=f
b = f < 8
theta = a==b  # theta
resBig.shape

In [ ]:
res = res**2

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML
title = raw.columns

fig,ax=plt.subplots()
coh_alpha = res[alpha,:,:].mean(axis=0)
coh_alpha = pd.DataFrame(coh_alpha, index=title, columns=title)
sns.heatmap(coh_alpha, cmap = "coolwarm", vmin=0,vmax=1.0)
plt.title("alpha(8-13Hz)")

fig,ax=plt.subplots()
coh_beta = res[beta,:,:].mean(axis=0)
coh_beta = pd.DataFrame(coh_beta, index=title, columns=title)
sns.heatmap(coh_beta, cmap = "coolwarm", vmin=0,vmax=1.0)
plt.title("beta(13-26Hz)")

fig,ax=plt.subplots()
coh_gamma = res[gamma,:,:].mean(axis=0)
coh_gamma = pd.DataFrame(coh_gamma, index=title, columns=title)
sns.heatmap(coh_gamma, cmap = "coolwarm", vmin=0,vmax=1.0)
plt.title("gamma(26-30Hz)")

fig,ax=plt.subplots()
coh_delta = res[delta,:,:].mean(axis=0)
coh_delta = pd.DataFrame(coh_delta, index=title, columns=title)
sns.heatmap(coh_delta, cmap = "coolwarm", vmin=0,vmax=1.0)
plt.title("delta(0.5-4Hz)")

fig,ax=plt.subplots()
coh_theta = res[theta,:,:].mean(axis=0)
coh_theta = pd.DataFrame(coh_theta, index=title, columns=title)
sns.heatmap(coh_theta, cmap = "coolwarm", vmin=0,vmax=1.0)
plt.title("theta(4-8Hz)")


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
ims = []
# fig = plt.figure()
fig,ax = plt.subplots()

plt.suptitle("alpha(8-13Hz)")
for i,res in enumerate(resBig):
    
    coh_alpha = res[alpha,:,:].mean(axis=0)
    coh_alpha = pd.DataFrame(coh_alpha, index=title, columns=title)
    # label = ax.set_title(f"alpha(8-13Hz):{i}epoch")
    label = ax.text(2,-1,f"alpha(8-13Hz):{i}epoch")
    im = ax.imshow(coh_alpha, cmap = "coolwarm", vmin=0,vmax=1.0)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)
    
    ims.append([im]+[label])

ani = animation.ArtistAnimation(fig, ims)
ani.save('alpha.mp4', writer="ffmpeg",dpi=100)


In [ ]:
ims = []
# fig = plt.figure()
fig,ax = plt.subplots()

plt.suptitle("beta(13-26Hz)")
for i,res in enumerate(resBig):
    
    coh_beta = res[beta,:,:].mean(axis=0)
    coh_beta = pd.DataFrame(coh_beta, index=title, columns=title)
    label = ax.text(2,-1,f"beta(13-26Hz):{i}epoch")
    im = ax.imshow(coh_beta, cmap = "coolwarm", vmin=0,vmax=1.0)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)
    
    ims.append([im]+[label])

ani = animation.ArtistAnimation(fig, ims)
ani.save('beta.mp4', writer="ffmpeg",dpi=100)


In [ ]:
ims = []
# fig = plt.figure()
fig,ax = plt.subplots()

plt.suptitle("gamma(26-30Hz)")
for i,res in enumerate(resBig):
    
    coh_gamma = res[gamma,:,:].mean(axis=0)
    coh_gamma = pd.DataFrame(coh_gamma, index=title, columns=title)
    label = ax.text(2,-1,f"gamma(26-30Hz):{i}epoch")
    im = ax.imshow(coh_gamma, cmap = "coolwarm", vmin=0,vmax=1.0)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)
    
    ims.append([im]+[label])

ani = animation.ArtistAnimation(fig, ims)
ani.save('gamma.mp4', writer="ffmpeg",dpi=100)


In [ ]:
ims = []
# fig = plt.figure()
fig,ax = plt.subplots()

plt.suptitle("delta(0.5-4Hz)")
for i,res in enumerate(resBig):
    
    coh_delta = res[delta,:,:].mean(axis=0)
    coh_delta = pd.DataFrame(coh_delta, index=title, columns=title)
    label = ax.text(2,-1,f"delta(0.5-4Hz):{i}epoch")
    im = ax.imshow(coh_delta, cmap = "coolwarm", vmin=0,vmax=1.0)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)
    
    ims.append([im]+[label])

ani = animation.ArtistAnimation(fig, ims)
ani.save('delta.mp4', writer="ffmpeg",dpi=100)

In [ ]:
ims = []
# fig = plt.figure()
fig,ax = plt.subplots()

plt.suptitle("theta(4-8Hz)")
for i,res in enumerate(resBig):
    
    coh_theta = res[theta,:,:].mean(axis=0)
    coh_theta = pd.DataFrame(coh_theta, index=title, columns=title)
    label = ax.text(2,-1,f"theta(4-8Hz):{i}epoch")
    im = ax.imshow(coh_theta, cmap = "coolwarm", vmin=0,vmax=1.0)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)
    
    ims.append([im]+[label])

ani = animation.ArtistAnimation(fig, ims)
ani.save('theta.mp4', writer="ffmpeg",dpi=100)

# 高齢者歩行データ解析

In [ ]:
emg_file_path = glob("../data/Elderly/EMG/sub01/*.mat")
emg_file_path.sort()
print(emg_file_path)

eeg_file_path = glob("../data/Elderly/EEG/sub01/*.edf")
eeg_file_path.sort()
eeg_file_path

['../data/Elderly/EMG/sub01/RAS100.mat', '../data/Elderly/EMG/sub01/RAS110.mat', '../data/Elderly/EMG/sub01/RAS90.mat', '../data/Elderly/EMG/sub01/noRAS1.mat', '../data/Elderly/EMG/sub01/noRAS2.mat']


['../data/Elderly/EEG/sub01/01_Matsumura_RAS100_Segment_0.edf',
 '../data/Elderly/EEG/sub01/01_Matsumura_RAS110_Segment_0_Segment_0.edf',
 '../data/Elderly/EEG/sub01/01_Matsumura_RAS90_Segment_0_Segment_0.edf',
 '../data/Elderly/EEG/sub01/01_Matsumura_noRAS1_Segment_0_Segment_0.edf',
 '../data/Elderly/EEG/sub01/01_Matsumura_noRAS2_Segment_0.edf',
 '../data/Elderly/EEG/sub01/01_Matsumura_standing_Segment_0.edf']

In [ ]:
emg_noRAS = make_emg_obj(emg_file_path[-2])
emg_noRAS2 = make_emg_obj(emg_file_path[-1])

In [ ]:
# emg_noRAS_raw, event_idx = prep_emg(emg_noRAS, h_freq=5/(1000/2),l_freq=200/(1000/2),Wn=4,btype="bandpass")

In [ ]:
montage = make_montage("../data/elect_loc64_2.elc")
eeg_noRAS = read_eeg(fname=eeg_file_path[-3], montage=montage)
#eeg_noRAS.plot()
eeg_noRAS2 = read_eeg(eeg_file_path[-2], montage)
#eeg_noRAS2.plot()
eeg_stand = read_eeg(eeg_file_path[-1],montage)
#eeg_stand.plot()

Extracting EDF parameters from /Users/bmhi/Desktop/research/data/Elderly/EEG/sub01/01_Matsumura_noRAS1_Segment_0_Segment_0.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/bmhi/Desktop/research/data/Elderly/EEG/sub01/01_Matsumura_noRAS2_Segment_0.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/bmhi/Desktop/research/data/Elderly/EEG/sub01/01_Matsumura_standing_Segment_0.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


 /var/folders/88/tbrdnkf549bg18p8s3rbscgm0000gp/T/ipykernel_36122/988844099.py:55: RuntimeWarning:Fiducial point nasion not found, assuming identity unknown to head transformation
 /var/folders/88/tbrdnkf549bg18p8s3rbscgm0000gp/T/ipykernel_36122/988844099.py:56: RuntimeWarning:Fiducial point nasion not found, assuming identity unknown to head transformation
 /var/folders/88/tbrdnkf549bg18p8s3rbscgm0000gp/T/ipykernel_36122/988844099.py:55: RuntimeWarning:Fiducial point nasion not found, assuming identity unknown to head transformation
 /var/folders/88/tbrdnkf549bg18p8s3rbscgm0000gp/T/ipykernel_36122/988844099.py:56: RuntimeWarning:Fiducial point nasion not found, assuming identity unknown to head transformation
 /var/folders/88/tbrdnkf549bg18p8s3rbscgm0000gp/T/ipykernel_36122/988844099.py:55: RuntimeWarning:Fiducial point nasion not found, assuming identity unknown to head transformation
 /var/folders/88/tbrdnkf549bg18p8s3rbscgm0000gp/T/ipykernel_36122/988844099.py:56: RuntimeWarning:Fi

In [ ]:
eeg_noRAS, emg_noRAS = add_event(eeg_noRAS, emg_noRAS)
eeg_noRAS2, emg_noRAS2 = add_event(eeg_noRAS2, emg_noRAS2)
eeg_noRAS.crop(10,70)
eeg_noRAS2.crop(10,70)
eeg_stand.crop(10,)

Creating RawArray with float64 data, n_channels=1, n_times=131199
    Range : 0 ... 131198 =      0.000 ...   131.198 secs
Ready.
Reading 0 ... 131198  =      0.000 ...   131.198 secs...
Creating RawArray with float64 data, n_channels=1, n_times=159649
    Range : 0 ... 159648 =      0.000 ...   159.648 secs
Ready.
Reading 0 ... 159648  =      0.000 ...   159.648 secs...


<RawEDF | 01_Matsumura_standing_Segment_0.edf, 64 x 55000 (55.0 s), ~88 kB, data not loaded>

In [ ]:
# eeg_noRAS.plot()

Using qt as 2D backend.


: 

: 

In [ ]:
# eeg_noRAS.plot(duration=20, scalings=dict(eeg=100e-6))
# eeg_stand = read_eeg(eeg_file_path[-1],montage)
# eeg_stand.plot()

振幅を確認すると、EOGが立位時におよそ3倍ほどの振幅で、EEGに関してはおよそ10倍の振幅であることがわかる。
また、全ての電極に一様のノイズが混入しているように見える。

まずは、このノイズについて、周期的なものかを確認するために、自己相関を算出する。
[自己相関に関するQiita記事](https://qiita.com/MToyokura/items/8a58cb43e634e6421834)

全ての電極で一様にノイズが確認されたことから、代表としてFp1の電極から、自己相関を計算する。

初めの10秒（10000ms）は振幅が大きく、歩行動作が始まっていない可能性があるので、除外した。

In [ ]:
# eeg_raw = eeg_noRAS.get_data()
# eeg_raw.shape
# import statsmodels.graphics.api as smg
# from statsmodels.graphics.tsaplots import plot_acf
# eeg_i = eeg_raw[0,100000:]
# fig,ax = plt.subplots(figsize=(15,6))
# plot_acf(eeg_i,lags=10000, ax=ax,title="noRAS_Fp1_correlogram")

コレログラムを見ると、自己相関はあまり高くないものの、周期性があり、その周期もおよそ500msと歩行周期の約半分であるように見える。

そのため、歩行に起因するアーチファクトである可能性がある。


次に、この異常な振幅の大きさがどの周波数によるものかを確認するために、FFTによるスペクトルパワープロットを確認する。

In [ ]:
# fig,ax = plt.subplots(2,1,figsize=(10,9))
# eeg_noRAS.plot_psd(fmax = 100, ax=ax[0],show=False)
# eeg_stand.plot_psd(fmax = 100, ax=ax[1]);

歩行条件では、全ての電極で、周波数構造が類似していることがわかる。

In [ ]:
# fig,ax = plt.subplots(3,1,figsize=(10,12))
# eeg_noRAS.plot_psd(fmax = 100, ax=ax[0],show=False)
# eeg_noRAS2.plot_psd(fmax = 100, ax=ax[1],show=False)
# eeg_stand.plot_psd(fmax = 100, ax=ax[2]);

EEGのレファレンスをチャンネル平均にすることで、全てのチャンネルに存在するアーチファクトの影響を除外する。

In [ ]:
eeg_noRAS.plot()
eeg_noRAS_preref = eeg_noRAS.copy()
eeg_noRAS.set_eeg_reference(ref_channels='average', projection=True)
eeg_noRAS.plot()

全体平均のリファレンスとすることで、全体にかかっているノイズに関しては除外ができた。
自己相関を確認したところ、自己相関自体も軽減したが、周期性に関しては変わらず見られるように思う。

In [ ]:
eeg_raw = eeg_noRAS.get_data()
eeg_raw.shape
import statsmodels.graphics.api as smg
from statsmodels.graphics.tsaplots import plot_acf
eeg_i = eeg_raw[0]
fig,ax = plt.subplots(figsize=(15,6))
plot_acf(eeg_i,lags=10000, ax=ax,title="noRAS_Fp1_correlogram")

続いて、パワースペクトル密度を計算したが、こちらは、レファレンス前と同じ構造である。これは、全体的な周波数の振幅構造が変化せず、スケールを変換しただけであることから生じていると考えられる。

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(10,12))
eeg_noRAS.plot_psd(fmax = 100, ax=ax[0],show=False)
eeg_noRAS2.plot_psd(fmax = 100, ax=ax[1],show=False)
eeg_stand.plot_psd(fmax = 100, ax=ax[2]);

エポッキングして、全体的な傾向がどのようになっているのかを確認する。

# 前処理
再参照は行なっているので、ここでは、ICA、ASRとフィルタリングについて検証する。